In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 22.7 MB/s eta 0:00:00


#Importing and Data Preprocessing

In [3]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

from scipy.stats import randint as sp_randint

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.DataStructs import BitVectToText

In [4]:
file_path = '/content/drive/My Drive/Datasets/AKT1_smiles_3.csv' ##Replace with your own paths
df = pd.read_csv(file_path)
file_path = '/content/drive/My Drive/Datasets/AKT1 CHEMBL.xlsx' ##Replace with your own paths
df1 = pd.read_excel(file_path)

In [5]:
def label_fromIC50(string):
  if string>6:
    return 1.0
  return 0.0

In [6]:
df['IC50'] = df1['pChEMBL Value']
df['Labels'] = df['IC50'].apply(label_fromIC50)
df=df.dropna()
#df_features = df[['QED','Docking Score','SAScore']].copy()
df_labels   = df['Labels'].copy()

#Morgan Fingerprints

In [7]:
def smiles_to_morgan_fingerprint(smiles, radius=2, nBits=2048):

    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(
            mol,
            radius=radius,
            nBits=nBits
        )
        return BitVectToText(fingerprint)

    except Exception:
        return None

In [8]:
df['Morgan_FP_2048'] = df['Smiles'].astype(str).apply(smiles_to_morgan_fingerprint)

df_clean = df.dropna(subset=['Morgan_FP_2048'])

fingerprint_matrix = np.array(
    [list(map(int, fp)) for fp in df_clean['Morgan_FP_2048']]
)

print(f"\n### Fingerprint Matrix for ML (NumPy Array) ###")
print(f"Shape of FP Matrix: {fingerprint_matrix.shape}")
print(f"First 10 bits of the first valid compound (Ethanol):\n{fingerprint_matrix[0, :10]}")

Streaming output truncated to the last 5000 lines.
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:21:31] DEPRECATION WARNING: please use MorganGenerator
[15:2


### Fingerprint Matrix for ML (NumPy Array) ###
Shape of FP Matrix: (5923, 2048)
First 10 bits of the first valid compound (Ethanol):
[0 0 0 0 0 0 0 0 0 0]


#Model Training

In [9]:
X = fingerprint_matrix
y = df_labels.to_numpy()

print(f"Features (X) shape: {X.shape}")
print(f"Labels (y) shape: {y.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

Features (X) shape: (5923, 2048)
Labels (y) shape: (5923,)

Training set size: 4738 samples
Testing set size: 1185 samples


In [22]:
model = RandomForestClassifier(
    n_estimators=182,
    random_state=42, # Set for reproducible results
    n_jobs=-1,         # Use all available cores for faster training
    criterion='entropy',
    min_samples_leaf=1
)

print("\nStarting model training...")
model.fit(X_train, y_train)
print("Model training complete.")


Starting model training...
Model training complete.


In [23]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_pred)

print("\n--- Model Evaluation ---")
print(f"Accuracy on Test Set: {accuracy:.4f}")
print(f"AUC Score: {auc_score}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


--- Model Evaluation ---
Accuracy on Test Set: 0.9283
AUC Score: 0.871899402847956

Classification Report:
              precision    recall  f1-score   support

         0.0       0.87      0.77      0.82       252
         1.0       0.94      0.97      0.96       933

    accuracy                           0.93      1185
   macro avg       0.91      0.87      0.89      1185
weighted avg       0.93      0.93      0.93      1185



#Hyperparam Tuning

In [13]:
# This dictionary defines the range/list of values to try for each hyperparameter.

param_dist = {
    "n_estimators": sp_randint(100, 200),
    "max_depth": [None],
    "min_samples_leaf": sp_randint(1, 5),
    "max_features": ['sqrt', 1.0],
    "criterion": ["entropy"]
}

In [18]:
# Initialize the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
# n_iter: number of parameter settings that are sampled (more is better but slower)
# cv: number of cross-validation folds (e.g., 5-fold cross-validation)
# scoring: the metric to optimize (e.g., 'accuracy', 'f1', 'roc_auc')

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=5,
    cv=2,
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

In [19]:
print("Starting Hyperparameter Tuning with Randomized Search...")
random_search.fit(X_train, y_train)
print("Tuning complete.")

Starting Hyperparameter Tuning with Randomized Search...
Fitting 2 folds for each of 5 candidates, totalling 10 fits
Tuning complete.


In [20]:
best_params = random_search.best_params_
best_score = random_search.best_score_
best_model = random_search.best_estimator_

print("--- Tuning Results ---")
print(f"Best Cross-Validation Score: {best_score:.4f}")
print("Best Parameters Found:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

--- Tuning Results ---
Best Cross-Validation Score: 0.9266
Best Parameters Found:
  criterion: entropy
  max_depth: None
  max_features: sqrt
  min_samples_leaf: 1
  n_estimators: 182


In [21]:
final_test_score = best_model.score(X_test, y_test)
print(f"\nAccuracy on FINAL Unseen Test Set: {final_test_score:.4f}")


Accuracy on FINAL Unseen Test Set: 0.9283
